In [25]:
import sys

sys.path.append('./src')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import requests
import os.path
from datetime import date, datetime, time
import time as pytime
from dataclasses import dataclass
from typing import Optional, Dict
import functools
import operator

import pandas as pd

import filters
from fields import FIELDS, FIELD_MAP
from time_block import TimeBlock
from parks_web import get_availability

# Display all rows
pd.set_option('display.max_rows', None)

In [67]:
fields = list(filters.fields_with_bases(FIELDS, 60))
#fields = [FIELD_MAP[key] for key in ['MB2', 'MB3']]
fields

[Field(key='Dahl2', id='1652', lighted=True, turf=False, bases=[60]),
 Field(key='Dahl3', id='1727', lighted=False, turf=False, bases=[60]),
 Field(key='Dahl4', id='1728', lighted=False, turf=False, bases=[60]),
 Field(key='MB2', id='1663', lighted=False, turf=False, bases=[60]),
 Field(key='MB3', id='1664', lighted=False, turf=False, bases=[60]),
 Field(key='Mag9', id='445', lighted=True, turf=True, bases=[60]),
 Field(key='LW3', id='2415', lighted=True, turf=False, bases=[60, 70, 80]),
 Field(key='LW4', id='2412', lighted=True, turf=False, bases=[60, 80]),
 Field(key='LW5', id='2413', lighted=True, turf=False, bases=[60, 80]),
 Field(key='LW6', id='2414', lighted=True, turf=False, bases=[60, 70, 80]),
 Field(key='LH2', id='1801', lighted=True, turf=True, bases=[60]),
 Field(key='VR2', id='1738', lighted=False, turf=False, bases=[60]),
 Field(key='Maple1', id='1700', lighted=False, turf=False, bases=[60]),
 Field(key='Maple2', id='1764', lighted=False, turf=False, bases=[60]),
 Field(

In [68]:
blocks = functools.reduce(operator.concat, [get_availability(field, '2025-05-20', '2025-06-20') for field in fields], [])
blocks = filter(None, map(filters.clip_range_date, blocks))
blocks = filters.filter_duration(blocks, 1)
blocks = list(filter(TimeBlock.is_weekday, blocks))
blocks

[Dahl2 Fri 2025-05-23: 05:00PM-10:00PM,
 Dahl2 Fri 2025-05-30: 08:00PM-10:00PM,
 Dahl2 Mon 2025-06-02: 08:00PM-10:00PM,
 Dahl2 Tue 2025-06-03: 08:00PM-10:00PM,
 Dahl2 Fri 2025-06-06: 08:00PM-10:00PM,
 Dahl2 Fri 2025-06-13: 08:00PM-10:00PM,
 Dahl2 Thu 2025-06-19: 08:30PM-10:00PM,
 Dahl2 Fri 2025-06-20: 05:00PM-10:00PM,
 Dahl3 Fri 2025-06-06: 07:00PM-09:00PM,
 Dahl4 Tue 2025-05-20: 07:30PM-09:00PM,
 Dahl4 Fri 2025-05-23: 05:00PM-09:00PM,
 Dahl4 Fri 2025-05-30: 07:30PM-09:00PM,
 Dahl4 Fri 2025-06-06: 05:00PM-09:00PM,
 Dahl4 Tue 2025-06-10: 06:00PM-09:00PM,
 Dahl4 Thu 2025-06-12: 06:00PM-09:00PM,
 Dahl4 Fri 2025-06-13: 05:00PM-09:00PM,
 Dahl4 Mon 2025-06-16: 05:00PM-09:00PM,
 Dahl4 Thu 2025-06-19: 07:30PM-09:00PM,
 MB2 Fri 2025-05-23: 05:45PM-09:00PM,
 MB3 Fri 2025-05-23: 05:45PM-09:00PM,
 Mag9 Fri 2025-05-23: 05:00PM-09:45PM,
 LW3 Fri 2025-05-23: 08:45PM-10:00PM,
 LW3 Tue 2025-06-03: 05:00PM-09:15PM,
 LW3 Wed 2025-06-04: 05:00PM-06:30PM,
 LW3 Thu 2025-06-05: 05:00PM-06:15PM,
 LW3 Tue 2025

In [70]:
df = pd.DataFrame(map(lambda b: b.to_json(), blocks))
df = df.sort_values('Date')
#df.to_csv('~/Downloads/lower_woodland.csv')
df[df['Day'].isin(['Thursday'])]

,Field,Date,Day,Start,End,Duration
40,LW6,2025-05-29,Thursday,05:00PM,06:15PM,1.25
46,LH2,2025-06-05,Thursday,07:00PM,09:45PM,2.75
24,LW3,2025-06-05,Thursday,05:00PM,06:15PM,1.25
14,Dahl4,2025-06-12,Thursday,06:00PM,09:00PM,3.00
56,VR2,2025-06-12,Thursday,05:00PM,06:30PM,1.50
27,LW3,2025-06-12,Thursday,05:00PM,06:15PM,1.25
36,LW5,2025-06-12,Thursday,05:00PM,08:00PM,3.00
59,VR2,2025-06-19,Thursday,05:00PM,09:00PM,4.00
17,Dahl4,2025-06-19,Thursday,07:30PM,09:00PM,1.50
29,LW3,2025-06-19,Thursday,06:15PM,08:00PM,1.75
